In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import pickle

In [7]:
features_df = pd.read_csv('extracted_features_pca.csv')

/tmp/ipykernel_2635014/220330452.py:1: DtypeWarning: Columns (9200,9201,9216,9217,9222,9223,9226,9227,9228,9229,9230,9231,9232,9233,9234,9235,9238,9239,9240,9241,9242,9243,9244,9245,9246,9247,9248,9249,9250,9251,9252,9253,9254,9255,9256,9257,9258,9259,9260,9261,9262,9263,9264,9265,9266,9267,9268,9269,9270,9271,9272,9273,9274,9275,9276,9277,9278,9279,9280,9281,9282,9283,9284,9285,9286,9287,9288,9289,9290,9291,9292,9293,9294,9295,9296,9297,9298,9299,9300,9301,9302,9303,9304,9305,9306,9307,9308,9309,9310,9311,9312,9313,9314,9315,9316,9317,9318,9319,9320,9321,9322,9323,9324,9325,9326,9327,9328,9329,9330,9331,9332,9333,9334,9335,9336,9337,9338,9339,9340,9341,9342,9343,9344,9345,9346,9347,9348,9349,9350,9351,9352,9353,9354,9355,9356,9357,9358,9359,9360,9361,9362,9363,9364,9365,9366,9367,9368,9369,9370,9371,9372,9373,9374,9375,9376,9377,9378,9379,9380,9381) have mixed types. Specify dtype option on import or set low_memory=False.
  features_df = pd.read_csv('extracted_features_pca.csv')


In [13]:

# Separate features, labels, and filenames
X = features_df.drop(columns=['label', 'filename']).select_dtypes(include=[np.number]).values
print(X.shape)

y = LabelEncoder().fit_transform(features_df['label'].values)
file_names = features_df['filename'].values


scaler = StandardScaler()
X = scaler.fit_transform(X)

(12600, 9222)


In [6]:
print(X.shape)

(12600, 9382)


In [14]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [15]:
best_rf_params = {
    "n_estimators": 1000,
    "min_samples_split": 10,
    "max_features":'sqrt',
    "max_depth":30,
    "random_state": 42,
    "n_jobs": -1

}

rf = RandomForestClassifier(**best_rf_params)
rf.fit(X_train, y_train)

# Test the model
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Model Accuracy: {accuracy:.4f}")

Random Forest Model Accuracy: 0.4718


edges shape: (50000,), orb_features shape: (4096,), hog_features shape: (3888,), lbp_features shape: (10,), color_histogram shape: (512,)

In [16]:
importances = rf.feature_importances_


# Split importances back into the respective feature categories
edges_importance = importances[:100]
orb_importance = importances[100:100+4096]
hog_importance = importances[100+4096:100+4096+3888]
lbp_importance = importances[100+4096+3888:100+4096+3888+30]
color_histogram_importance = importances[100+4096+3888+30:]

# Sum the importances for each category
category_importances = {
    'edges': np.sum(edges_importance),
    'orb_features': np.sum(orb_importance),
    'hog_features': np.sum(hog_importance),
    'lbp_features': np.sum(lbp_importance),
    'color_histogram': np.sum(color_histogram_importance)
}

# Convert to pandas Series for easier viewing
category_importances_series = pd.Series(category_importances)

# Print the importances by category
print(category_importances_series.sort_values(ascending=False))


color_histogram    0.360866
hog_features       0.326989
orb_features       0.263037
lbp_features       0.036908
edges              0.012200
dtype: float64


In [17]:
# Save the model to a file
model_filename = 'trained_random_forest_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(rf, file)